In [ ]:
import pandas as pd
import sqlite3

In [ ]:
import sqlite3
conn = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/anekdot-ru.db")
df = pd.read_sql_query("SELECT text FROM anecdot WHERE date BETWEEN '2020-01-01' and '2020-01-31'", conn) # WHERE date BETWEEN '2020-01-01' and '2020-01-31'
# Verify that result of SQL query is stored in the dataframe
print(df.head())

                                                text
0                   1 января - синий день календаря!
1  1 января. Сегодня ночью какой-то мужик всю стр...
2  Технологи завода "Кристал", доказали, что на у...
3  Нарядил ёлочку - перевесил мышку из холодильника.
4  Почему Дед Мороз 1 января не ходит с мешком ми...


In [ ]:
f = open('anecdot.txt', 'w')
for i in df['text']:
    f.write(i + '\n')

In [ ]:
!pip install transformers 

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загружаем модель
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

     |████████████████████████████████| 4.0 MB 12.2 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 42.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [ ]:
train_path = '/content/anecdot.txt'

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file /content/cached_lm_GPT2Tokenizer_64_anecdot.txt [took 0.056 s]


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(  
    output_dir="./finetuned", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=25, # number of training epochs
    per_device_train_batch_size=20, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/anekdyear.pth')

***** Running training *****
  Num examples = 15534
  Num Epochs = 25
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 16
  Total optimization steps = 1200


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/anekdyear.pth')

In [ ]:
text = "Звонит программист"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Звонит программист по мобильному телефону и жалуется:
— Я так и знал, что вы мне не перезвоните! Это у вас какой-то корпоративный телефон сломался!
Звонок в сервисный центр:
— Алло, это я, мастер! У меня тут сломался телефон! Я его починил, а вы мне звоните, чтобы мне починили!

Жена спрашивает мужа:
— Как дела? Как дела, муж?


